In [ ]:
%matplotlib inline
import random
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data as mnist_data
mnist = mnist_data.read_data_sets("MNIST_data/", one_hot=True)

In [ ]:
with tf.name_scope('inputs'):
    x = tf.placeholder(tf.float32, [None, 784], name='x')
    y = tf.placeholder(tf.float32, [None, 10], name='y')
    x_image = tf.reshape(x, [-1,28,28,1], name='image')

In [ ]:
with slim.arg_scope(
    [slim.conv2d, slim.fully_connected],
    biases_initializer=tf.constant_initializer(0.1),
    weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
):
    out = slim.conv2d(x_image, 32, [5, 5], scope='conv1')
    out = slim.max_pool2d(out, [2, 2], scope='pool1')
    out = slim.conv2d(out, 64, [5, 5], scope='conv2')
    out = slim.max_pool2d(out, [2, 2], scope='pool2')
    out = slim.flatten()
    out = slim.fully_connected(out, 128, scope = 'fc1')
    out = slim.fully_connected(out, 10, activation_fn=None, scope='fc2')

In [ ]:
with tf.name_scope('loss'):    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=out, labels=y), name='cross_entropy')
loss_summary = tf.summary.scalar('loss', loss)

with tf.name_scope('accuracy'):
    correct_prediction = tf.equal(tf.argmax(out, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

In [ ]:
ITERS = 10000
BATCH_SIZE = 1
LEARNING_RATE = 0.0001

optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
output_dir = 'logs/adam/'

train_step = slim.learning.create_train_op(loss, optimizer)
init = tf.global_variables_initializer()

config = tf.ConfigProto(log_device_placement = False)
config.gpu_options.allow_growth = True
config.gpu_options.allocator_type = 'BFC'
session = tf.Session(config=config)

session.run(init)
summary_writer = tf.summary.FileWriter(output_dir, sess.graph)

In [ ]:
for it in range(ITERS):
    
    batch_xs, batch_ys = mnist.train.next_batch(BATCH_SIZE)    
    _, loss_s = sess.run([train_step, loss_summary], feed_dict={
        x: batch_xs, y: batch_ys,
    })

    if it % 100 == 0:
        summary_writer.add_summary(loss_s, it)
        
    if it % 100 == 0:
        test_xs, test_ys = mnist.test.next_batch(128)
        acc, acc_s = sess.run([accuracy, accuracy_summary], feed_dict={
            x: test_xs, y: test_ys,
        })
        summary_writer.add_summary(acc_s, it)
        print("%5.2f" % (acc*100))

In [ ]:
final_acc = sess.run(accuracy, feed_dict={
    x: mnist.test.images,
    y: mnist.test.labels,
})
print('Accuracy : %5.2f %%' % (final_acc*100))

In [ ]:
def test_digit(sample):
    
    sample = sample[np.newaxis, ...]
    prediction = sess.run(tf.nn.softmax(out), feed_dict={x: sample})
    ans = np.argmax(prediction)

    fig = plt.figure(figsize=(12,4))

    ax = fig.add_subplot(1, 2, 1)
    ax.imshow(sample.transpose().reshape(28, 28), 
              cmap=matplotlib.cm.binary,interpolation ='none')
    plt.xticks(np.array([]))
    plt.yticks(np.array([]))

    ax = fig.add_subplot(1, 2, 2)
    bar_list = ax.bar(np.arange(10), prediction[0], align='center')
    bar_list[ans].set_color('g')
    ax.set_xticks(np.arange(10))
    ax.set_xlim([-1, 10])
    ax.grid('on')

    plt.show()

    print('Answer: {}'.format(ans))

In [ ]:
idx = random.randint(0, mnist.test.images.shape[0])
sample = mnist.test.images[idx,:]
test_digit(sample)
print(sample.shape)

In [ ]:
print('True Answer: {}'.format(np.argmax(mnist.test.labels[idx])))